In [1]:
# coding: utf-8
import requests
import time
import pandas as pd
import matplotlib.pyplot as plt

# 绘制滤波前后的时间序列图
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体为宋体或其他已安装的字体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题

PATH = 'D:/All About Learning/Quant/gate/'

In [2]:
def get_candle_sticks(pair: str, after: str = '', before: str = '', interval: str = '7d'):
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

    url = '/spot/candlesticks'
    query_param = f'currency_pair={pair}&interval={interval}'
    if after != '':
        query_param += f'&from={after}'
    if before != '':
        query_param += f'&to={before}'

    r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)
    return r.json()

In [69]:
btc = get_candle_sticks('BTC_USDT')
data = pd.DataFrame(btc)
print(data)

             0                    1         2         3         4         5  \
0   1639353600   685901491.92981140  46691.75  50183.44   45455.3  50058.12   
1   1639958400   494501836.71176106   50769.1  51816.08  45573.97  46683.15   
2   1640563200   759599244.50330850  47296.37  52079.14     45700   50769.1   
3   1641168000   892842910.31189970  41860.17  47564.23  40528.06  47296.36   
4   1641772800   522086714.12299454  43072.57  44418.93  39653.46  41868.74   
..         ...                  ...       ...       ...       ...       ...   
95  1696809600   729958792.27261250   27154.8   27987.8     26538   27919.3   
96  1697414400  1645821369.38518760   29991.5   30380.5   27114.6     27153   
97  1698019200  3360004588.73374560   34525.6   35427.4   29881.6   29991.5   
98  1698624000  1599896364.87064670   35011.9   35983.4   34035.7   34525.7   
99  1699228800   523199934.17492145   35300.4   35887.9   34523.1   35011.9   

                 6  
0   14463.37598711  
1   10051

根据与gateio的tradingview对照，返回的字段应该是：['时间戳', 'xxx', '收盘', '高点', '低点', '开盘', '成交量']

In [70]:
data.columns=['时间戳', '成交额', '收盘', '高点', '低点', '开盘', '成交量']
data.set_index('时间戳', inplace=True)

In [71]:
print(data)

                            成交额        收盘        高点        低点        开盘  \
时间戳                                                                       
1639353600   685901491.92981140  46691.75  50183.44   45455.3  50058.12   
1639958400   494501836.71176106   50769.1  51816.08  45573.97  46683.15   
1640563200   759599244.50330850  47296.37  52079.14     45700   50769.1   
1641168000   892842910.31189970  41860.17  47564.23  40528.06  47296.36   
1641772800   522086714.12299454  43072.57  44418.93  39653.46  41868.74   
...                         ...       ...       ...       ...       ...   
1696809600   729958792.27261250   27154.8   27987.8     26538   27919.3   
1697414400  1645821369.38518760   29991.5   30380.5   27114.6     27153   
1698019200  3360004588.73374560   34525.6   35427.4   29881.6   29991.5   
1698624000  1599896364.87064670   35011.9   35983.4   34035.7   34525.7   
1699228800   523199934.17492145   35300.4   35887.9   34523.1   35011.9   

                       成

In [3]:
def get_all_pairs():
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

    url = '/spot/currency_pairs'
    query_param = ''
    r = requests.request('GET', host + prefix + url, headers=headers)
    result = r.json()
    return {item['id']: item['sell_start'] for item in result}

In [4]:
import re

def remove_lever_pairs(pairs):
    pattern = re.compile(r'(\d+[SL])$')
    clean_pairs = {}
    # 去除X倍多/空情况
    for pair in pairs:
        token, _ = pair.split('_')
        if not re.search(pattern, token):
            clean_pairs[pair] = pairs[pair]

    return clean_pairs

In [5]:
def get_usdt_pairs(pairs):
    usdt_pairs = {}
    for pair in pairs:
        # 获取yy部分
        stable = pair.split('_')[1]
        if stable == 'USDT':
            usdt_pairs[pair] = pairs[pair]
    return usdt_pairs

In [6]:
def get_all_spot_usdt_pairs():
    return remove_lever_pairs(get_usdt_pairs(get_all_pairs()))

In [76]:
print(get_all_spot_usdt_pairs())

['10SET_USDT', '1ART_USDT', '1EARTH_USDT', '1INCH_USDT', '3KM_USDT', '4TOKEN_USDT', '88MPH_USDT', 'AAG_USDT', 'AART_USDT', 'AAVE_USDT', 'ABBC_USDT', 'ABT_USDT', 'ACA_USDT', 'ACE_USDT', 'ACH_USDT', 'ACM_USDT', 'ACN_USDT', 'ACS_USDT', 'ACX_USDT', 'ADAPAD_USDT', 'ADA_USDT', 'ADEL_USDT', 'ADP_USDT', 'ADS_USDT', 'ADX_USDT', 'AD_USDT', 'AERGO_USDT', 'AE_USDT', 'AFC_USDT', 'AGB_USDT', 'AGILITY_USDT', 'AGIX_USDT', 'AGI_USDT', 'AGLA_USDT', 'AGLD_USDT', 'AGS_USDT', 'AG_USDT', 'AIBB_USDT', 'AICODE_USDT', 'AIDOGE_USDT', 'AIEPK_USDT', 'AIE_USDT', 'AIOZ_USDT', 'AIRTNT_USDT', 'AIR_USDT', 'AISC_USDT', 'AIS_USDT', 'AITECH_USDT', 'AKITA_USDT', 'AKRO_USDT', 'AKT_USDT', 'AKV_USDT', 'ALAYA_USDT', 'ALA_USDT', 'ALCX_USDT', 'ALD_USDT', 'ALEPH_USDT', 'ALEX_USDT', 'ALGO_USDT', 'ALICE_USDT', 'ALI_USDT', 'ALN_USDT', 'ALPACA_USDT', 'ALPA_USDT', 'ALPHA_USDT', 'ALPHR_USDT', 'ALPH_USDT', 'ALPINE_USDT', 'ALTB_USDT', 'ALU_USDT', 'ALY_USDT', 'AMB_USDT', 'AMC_USDT', 'AME_USDT', 'AMKT_USDT', 'AMPL_USDT', 'AMP_USDT', 'AMS_

In [94]:
cs = get_candle_sticks('YGG_USDT', after='1620720000', interval='8h')
print(cs)

[]


In [11]:
def get_current_timestamp():
    timestamp = int(time.time())
    hour = timestamp // 3600 * 3600
    return hour

In [100]:
get_current_timestamp()

1699437600

In [106]:
candle_sticks = []
for t in range(1620720000, 1699437600, 3600*800):
    try:
        tmp = get_candle_sticks('YGG_USDT', after=t, interval='8h')
        print(len(tmp))
        candle_sticks += tmp
        time.sleep(1)
    except Exception as e:
        time.sleep(10)


0
0
67
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
34


In [108]:
df = pd.DataFrame(candle_sticks)
print(df)

               0                       1        2        3        4        5  \
0     1627459200     336713.363133588113   1.7488    2.238    0.179    0.179   
1     1627488000     611633.842362357654     1.42    1.808   1.3332   1.7393   
2     1627516800     835876.729138228632   1.0105   1.4591    0.902    1.427   
3     1627545600    1644828.425511968958    1.507   1.6378    0.995   1.0042   
4     1627574400  948395.368630649732182   2.1759    2.215     1.48   1.5013   
...          ...                     ...      ...      ...      ...      ...   
2496  1699315200        1312526.73154526  0.34574  0.35507  0.33082  0.34385   
2497  1699344000        1542710.70130229   0.3315   0.3688  0.33035   0.3458   
2498  1699372800        1331541.10026113  0.37333   0.3765   0.3267  0.33087   
2499  1699401600         949281.56094999  0.37995   0.3844  0.35631  0.37316   
2500  1699430400         195310.79924661  0.38288  0.38452  0.37249  0.38029   

                     6  
0      174953.

In [115]:
print(df.iloc[66:][:])

               0                 1        2        3        4        5  \
66    1699430400                 0   5.5935   5.5935   5.5935   5.5935   
67    1629360000  2010062.11201349   7.0145   7.6022   6.1232     6.33   
68    1629388800   941682.84652803   7.4191   7.4897   6.8988    7.016   
69    1629417600   917358.02307554   7.6184    7.967   7.3942   7.4116   
70    1629446400  1444712.27164156   8.2823   8.7935   7.5491   7.6184   
...          ...               ...      ...      ...      ...      ...   
2496  1699315200  1312526.73154526  0.34574  0.35507  0.33082  0.34385   
2497  1699344000  1542710.70130229   0.3315   0.3688  0.33035   0.3458   
2498  1699372800  1331541.10026113  0.37333   0.3765   0.3267  0.33087   
2499  1699401600   949281.56094999  0.37995   0.3844  0.35631  0.37316   
2500  1699430400   195310.79924661  0.38288  0.38452  0.37249  0.38029   

                     6  
66                   0  
67     289008.76945840  
68     129060.66020212  
69     1184

In [111]:
1627488000-1627459200

28800

In [112]:
3600*800

2880000

In [7]:
import json

# 读取 JSON 文件
with open('D:/All about learning/Quant/gate/usdt_pairs.json', 'r') as file:
    json_data = file.read()

# 将 JSON 数据解析为字典
data_dict = json.loads(json_data)

# 现在 data_dict 中包含了 JSON 文件中的数据，你可以使用它进行后续操作

In [12]:
for pair in data_dict:
    start = data_dict[pair]

    if start > 0:
        continue

    cur = get_current_timestamp()
    while start < cur:
        tmp = get_candle_sticks(pair, start)
        if len(tmp) > 0:
            break
        start += 100*8*60*60 # 100条 8h

    print(f'{pair}: start {start}')
    data_dict[pair] = start

KeyboardInterrupt: 

In [9]:
get_candle_sticks('BTC_USDT', '0')

[]